In [4]:
# import relevant dependencies
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas


In [3]:
# Loading the data in pandas dataframe
lending_df = pd.read_csv("lending_data.csv")
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [7]:
# Creating a label set y from "loan_status" column and features X Dataframe from the remaining.
X = lending_df.drop(columns = "loan_status")
y = lending_df["loan_status"]

In [17]:
# Display X DataFrame variable
X[: 5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


In [18]:
# Display y DataFrame variable
y[: 5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

Split the Data into Training and Testing Sets

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 7)

Create a Logistic Regression Model with the Original Data

In [9]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=500,
                                random_state=1)
classifier

LogisticRegression(max_iter=500, random_state=1)

 Fit (train) or model using the training data

In [10]:
 classifier.fit(X_train, y_train)

LogisticRegression(max_iter=500, random_state=1)

 Score the model using the test data

In [11]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9914878250103177
Testing Data Score: 0.9924164259182832


In [12]:
# MAKE PREDICTIONS
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


Calculate the Accuracy Score

In [13]:
from sklearn.metrics import accuracy_score
# Display the accuracy score for the test dataset.
accuracy_score(y_test, predictions)

0.9924164259182832

 Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[18679,    80],
       [   67,   558]], dtype=int64)

 Classification Report

In [15]:
from sklearn.metrics import classification_report
target_names = ["Class Purple", "Class Yellow"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

Class Purple       1.00      1.00      1.00     18759
Class Yellow       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



Question: How well does the logistic regression model predict both the 0 (healthy loan) and 1 (high-risk loan) labels?

Answer: In general the model does  a good job at predicting healthy and high-risk loans. However, the model predicts healthy loans better than it does high-risk.

Predict a Logistic Regression Model with Resampled Training Data
Use the RandomOverSampler module from the imbalanced-learn library to resample the data. 
Be sure to confirm that the labels have an equal number of data points.

In [19]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
oversampler_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_os_model, y_os_model = oversampler_model.fit_resample(X_train, y_train)

In [21]:
# Display distinct count of the resampled label data
y_os_model.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

 Use the LogisticRegression classifier and the resampled data to fit the model and make predictions

In [22]:
# Instantiate the Logistic Regression model by assigning a random_state parameter of 1 to the model
logistic_os_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
logistic_os_model.fit(X_os_model, y_os_model)

# Make a prediction using the testing data
logistic_os_predict = logistic_os_model.predict(X_test)

In [24]:
# Print the accuracy score of the model 
score = accuracy_score(y_test, logistic_os_predict)
print(f" Accuracy Score: {score}")

 Accuracy Score: 0.9952022286421791


In [26]:
# Generate a confusion matrix for the model
cm_os = confusion_matrix(y_test, logistic_os_predict)
print(" Confusion Matrix of Resampled Data: ")
print(cm_os)

 Confusion Matrix of Resampled Data: 
[[18668    91]
 [    2   623]]


In [27]:
# Print the classification report for the model
report = classification_report(y_test, logistic_os_predict)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



Question: How well does the logistic regression model, fit with oversampled data, predict both the 0 (healthy loan) and 1 (high-risk loan) labels?

Answer: The model"s prediction is almost identically to the above model, with this model also predicting healthy loans more accurately than high-risk loans.